# 심층 컨볼루션 신경망(DCNN)

## 텐서플로 2.x의 ConvNets

In [8]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


### with MaxPooling

In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


## DCNN의 예: LeNet

### 텐서플로 2.x에서 LeNet 코드 예제

In [38]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers

%load_ext tensorboard
# 로그 초기화
#!rm -rf ./logs/

# 신경망과 훈련
epochs = 20
batch_size = 128
verbose = 1
optimizer = tf.keras.optimizers.Adam()
validation_split = 0.9

img_rows, img_cols = 28, 28  # 입력 이미지 차원
input_shape = (img_rows, img_cols, 1)
nb_classes = 10  # 출력의 개수 = 자리 수

# convnet 정의
def build(input_shape, classes):
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(20, (5, 5), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # 소프트맥스 본류기
    model.add(layers.Dense(classes, activation='softmax'))
    return model

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [33]:
# 데이터 : 훈련과 테스트 집합 사이에 섞고 분할
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

# 크기 조정
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

# 정규화
X_train, X_test = X_train/255.0, X_test/255.0

# 형식 변환
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 한 줄로 가능
#X_train = (X_train.reshape((60000, 28, 28, 1))/255.0).astype('float32')
#X_test = (X_test.reshape((10000, 28, 28, 1))/255.0).astype('float32')

# 부류 벡터를 이진 부류 행렬로 변환
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

In [34]:
# 최적화기와 모델 초기화
model = build(input_shape=input_shape, classes=nb_classes)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               400500    
_________________________________________________________________
dense_9 (Dense)              (None, 10)               

In [13]:
# 텐서보드를 사용
callbacks = [
    # 텐서보드 로그를 './logs' 디렉터리에 작성
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [40]:
# 적합화
history = model.fit(X_train, y_train,
                   batch_size=batch_size, epochs=20,
                   verbose=verbose, validation_split=validation_split,
                   callbacks=callbacks)

score = model.evaluate(X_test, y_test, verbose=verbose)
print("\nTest score:", score[0])
print("Test Accuracy:", score[1])

In [2]:
%load_ext tensorboard
%tensorboard --logdir logs

In [42]:
100 - 0.21

99.79

## 딥러닝으로 CIFAR-10 이미지 인식

In [11]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
%load_ext tensorboard
# 로그 초기화
#!rm -rf ./logs/

# CIFAR-10은 3채널 32x32 픽셀의 6만개 이미지다.
img_channels = 3
img_rows = 32
img_cols = 32

# 상수
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

# convnet 정의
def build(input_shape, classes):
    model = models.Sequential()
    model.add(layers.Convolution2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes, activation='softmax'))
    return model

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
# 데이터 : 훈련과 테스트 집합 사이에 섞고 분할
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0

# 부류 벡터를 이진 부류 행렬로 변환
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

170500096/170498071 [==============================] - 68s 0us/step


In [9]:
X_train.shape, X_train.dtype

((50000, 32, 32, 3), dtype('float64'))

In [12]:
model=build((img_rows, img_cols, img_channels), CLASSES)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3686912   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [5]:
# 텐서보드를 사용하라.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [14]:
# 훈련
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
         epochs=EPOCHS, validation_split=VALIDATION_SPLIT,
         verbose=VERBOSE, callbacks=callbacks)
score = model.evaluate(X_test, y_test,
                      batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score:", score[0])
print("Test Accuracy:", score[1])

Epoch 1/20
313/313 [==============================] - 20s 64ms/step - loss: 1.7006 - accuracy: 0.3972 - val_loss: 1.3622 - val_accuracy: 0.5259
Epoch 2/20
313/313 [==============================] - 20s 64ms/step - loss: 1.3665 - accuracy: 0.5153 - val_loss: 1.2269 - val_accuracy: 0.5776
Epoch 3/20
313/313 [==============================] - 20s 63ms/step - loss: 1.2326 - accuracy: 0.5677 - val_loss: 1.2363 - val_accuracy: 0.5718
Epoch 4/20
313/313 [==============================] - 20s 63ms/step - loss: 1.1483 - accuracy: 0.5990 - val_loss: 1.1925 - val_accuracy: 0.5900
Epoch 5/20
313/313 [==============================] - 20s 64ms/step - loss: 1.0692 - accuracy: 0.6262 - val_loss: 1.1218 - val_accuracy: 0.6137
Epoch 6/20
313/313 [==============================] - 20s 65ms/step - loss: 1.0027 - accuracy: 0.6515 - val_loss: 1.0708 - val_accuracy: 0.6342
Epoch 7/20
313/313 [==============================] - 21s 66ms/step - loss: 0.9403 - accuracy: 0.6743 - val_loss: 1.0276 - val_accuracy:

In [4]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6009 (pid 59118), started 20:00:52 ago. (Use '!kill 59118' to kill it.)

### 심층 신경망으로 CIFAR-10 성능 향상
- Conv2D or Convolution2D -> padding='valid' : 입력과 필터가 완전히 겹치는 경우에만 계산, 출력이 입력보다 작음을 의미
- Conv2D or Convolution2D -> padding='same' : 출력이 입력과 동일한 크기로 입력 주변 영역에 0이 채워짐

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
import numpy as np

EPOCHS = 50
NUM_CLASSES = 10
BATCH_SIZE = 128

def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # 정규화
    mean = np.mean(x_train, axis=(0,1,2,3))
    std = np.std(x_train, axis=(0,1,2,3))
    x_train = (x_train - mean) / (std + 1e-7)
    x_test = (x_test - mean) / (std + 1e-7)
    
    y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)
    
    return x_train,y_train,x_test,y_test

In [28]:
(x_train,y_train,x_test,y_test) = load_data()
x_train.shape,x_train.dtype,y_train.shape,y_train.dtype

((50000, 32, 32, 3), dtype('float32'), (50000, 10), dtype('float32'))

In [32]:
# 모델 강화
def build_model():
    
    model = models.Sequential()
    
    # 첫 번째 블록
    model.add(layers.Conv2D(32, (3,3), padding='same',
                           input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    
    # 두 번째 블록
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    
    # 세 번째 블록
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    
    # 밀집 출력 계층
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    # 모델 요약
    model.summary()
    
    return model

In [33]:
model = build_model()
# 모델 컴파일
model.compile(loss='categorical_crossentropy',
             optimizer='RMSprop',
             metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 64)       

In [36]:
# 텐서보드를 사용하라.
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

In [38]:
# 훈련
batch_size = 64
model.fit(x_train, y_train, batch_size=batch_size,
         epochs=EPOCHS, validation_data=(x_test, y_test),
         verbose=1, callbacks=callbacks)
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print("Test Accuracy:", score[1])

Epoch 1/50
782/782 [==============================] - 149s 191ms/step - loss: 1.7504 - accuracy: 0.4755 - val_loss: 1.4732 - val_accuracy: 0.5798
Epoch 2/50
782/782 [==============================] - 148s 189ms/step - loss: 1.0791 - accuracy: 0.6529 - val_loss: 1.0840 - val_accuracy: 0.6829
Epoch 3/50
782/782 [==============================] - 148s 189ms/step - loss: 0.8493 - accuracy: 0.7146 - val_loss: 0.8506 - val_accuracy: 0.7197
Epoch 4/50
782/782 [==============================] - 147s 189ms/step - loss: 0.7239 - accuracy: 0.7518 - val_loss: 0.7783 - val_accuracy: 0.7441
Epoch 5/50
782/782 [==============================] - 148s 189ms/step - loss: 0.6461 - accuracy: 0.7775 - val_loss: 0.6709 - val_accuracy: 0.7752
Epoch 6/50
782/782 [==============================] - 152s 194ms/step - loss: 0.5914 - accuracy: 0.7983 - val_loss: 0.6001 - val_accuracy: 0.7980
Epoch 7/50
782/782 [==============================] - 151s 193ms/step - loss: 0.5415 - accuracy: 0.8142 - val_loss: 0.5985 -

In [ ]:
# 디스크에 저장
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

In [3]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6009 (pid 59118), started 20:00:44 ago. (Use '!kill 59118' to kill it.)

### 데이터 보강으로 CIFAR-10 성능 개선

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 이미지 보강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [42]:
x_train.shape, x_train.dtype

((50000, 32, 32, 3), dtype('float32'))

In [40]:
# 학습
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=EPOCHS,
                    verbose=1, validation_data=(x_test,y_test),
                   callbacks=callbacks)

In [ ]:
# 디스크에 저장
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

In [ ]:
# 테스트
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

### CIFAR-10 으로 예측

In [2]:
from tensorflow import keras as k
import numpy as np
import scipy.misc

from skimage.transform import resize
from imageio import imread

from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import SGD

model = k.models.load_model('model/model_gnr.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [3]:
img_names = ["images/cat-standing.jpg", "images/dog.jpg"]
imgs = [np.transpose(resize(imread(img_name), (32, 32)), (2, 0, 1)).astype("float32") 
    for img_name in img_names]
#imgs = [resize(imread(img_name), (32, 32)).astype("float32") for img_name in img_names]
imgs = np.array(imgs) / 255
print("imgs.shape:", imgs.shape)

imgs.shape: (2, 3, 32, 32)


In [6]:
optim = SGD()
model.compile(loss="categorical_crossentropy", optimizer=optim, metrics=["accuracy"])

predictions = model.predict(imgs)
print("predictions:", predictions)

'optim = SGD()\nmodel.compile(loss="categorical_crossentropy", optimizer=optim, metrics=["accuracy"])\n\npredictions = model.predict(imgs)\nprint("predictions:", predictions)'